In [4]:
import tkinter as tk
from tkinter import messagebox
import tkintermapview
import pandas as pd
import numpy as np
import ast
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Adatok betöltése és előfeldolgozása
import pandas as pd
import numpy as np
import ast
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

df = pd.read_csv("output.csv", sep="\t")
df['Coordinates'] = df['Coordinates'].apply(ast.literal_eval)

# Szélességi és hosszúsági koordináták kinyerése
df['lat'] = df['Coordinates'].apply(lambda coord: coord[0])
df['lon'] = df['Coordinates'].apply(lambda coord: coord[1])

# Független és függő változók definiálása
X = df[['lat', 'lon']]
y = df['fastest_mode']

# Train-test felosztás az adatmennyiség korlátozottsága miatt fontos
# 70%-30%-os arány választva a limitált adat miatt a modell valós prediktív teljesítményének jobb megítéléséhez
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Random Forest Classifier paraméterek bővítése GridSearchCV-vel
param_grid = {
    'n_estimators': [50, 100, 200], # döntési fák száma
    'max_depth': [None, 10, 20, 30], # fák mélységének korlátozása
    'min_samples_split': [2, 5, 10], # minimum mintaszám egy csomópont felosztásához
    'min_samples_leaf': [1, 2, 4], # minimum mintaszám a levélben
    'bootstrap': [True, False] # bootstrap mintavétel alkalmazása
}

# GridSearchCV használata a legjobb paraméterek automatikus megtalálásához
grid_search = GridSearchCV(estimator=RandomForestClassifier(random_state=42),
                           param_grid=param_grid,
                           cv=5,  # ötszörös keresztvalidáció
                           n_jobs=-1,  # párhuzamosítás a számítások gyorsításához
                           verbose=2)  # részletes információk a futtatás során

# Modell illesztése a legjobb paraméterek megtalálásához
grid_search.fit(X_train, y_train)

# A legjobb modell kiválasztása
best_clf = grid_search.best_estimator_
print(f"Best parameters: {grid_search.best_params_}")

# Modell predikciójának kiértékelése tesztadatokon
y_pred = best_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Test accuracy: {accuracy:.2f}")

# Bővebb kiértékelési eredmények
print("Classification Report:\n", classification_report(y_test, y_pred))
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))


def predict_transport_mode(coordinate):
    """
    Predict the best transportation mode for a given coordinate.
    
    Parameters:
      coordinate (tuple): A tuple (lat, lon) of the destination.
    
    Returns:
      mode (str): Predicted best transportation mode.
      confidence (float): Confidence of the prediction.
    """
    X_new = pd.DataFrame([coordinate], columns=['lat', 'lon'])
    pred_mode = best_clf.predict(X_new)[0]
    pred_proba = best_clf.predict_proba(X_new)[0]
    confidence = np.max(pred_proba)
    return pred_mode, confidence

# Build the Interactive Map Window
root = tk.Tk()
root.title("Select Destination on Map")
root.geometry("800x600")

selection_enabled = False

def enable_selection():
    global selection_enabled
    selection_enabled = True
    select_button.config(text="Click on Map...")

select_button = tk.Button(root, text="Select Location", command=enable_selection)
select_button.pack(pady=5)

map_widget = tkintermapview.TkinterMapView(root, width=800, height=550, corner_radius=0)
map_widget.set_position(47.5316, 21.6273)  # Coordinates for Debrecen
map_widget.set_zoom(12)
map_widget.pack(fill="both", expand=True)

def on_map_click(event):
    global selection_enabled
    if not selection_enabled:
        return

    map_widget.delete_all_marker()

    lat, lon = map_widget.convert_canvas_coords_to_decimal_coords(event.x, event.y)
    mode, conf = predict_transport_mode((lat, lon))
    
    marker_text = f"{mode} ({conf:.2f})"
    map_widget.set_marker(lat, lon, text=marker_text)
    
    selection_enabled = False
    select_button.config(text="Select Location")

map_widget.canvas.bind("<Button-1>", on_map_click)

root.mainloop()

Fitting 5 folds for each of 216 candidates, totalling 1080 fits
Best parameters: {'bootstrap': True, 'max_depth': None, 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 200}
Test accuracy: 0.96
Classification Report:
               precision    recall  f1-score   support

   bicycling       0.94      0.92      0.93       287
     driving       0.93      0.90      0.91       412
       drone       0.97      0.98      0.98      2301

    accuracy                           0.96      3000
   macro avg       0.95      0.93      0.94      3000
weighted avg       0.96      0.96      0.96      3000

Confusion Matrix:
 [[ 264    1   22]
 [   3  372   37]
 [  15   29 2257]]


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\PS User\AppData\Roaming\Python\Python311\site-packages\tkintermapview\map_widget.py", line 776, in mouse_move
    mouse_move_x = self.last_mouse_down_position[0] - event.x
                   ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^
TypeError: 'NoneType' object is not subscriptable
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\ProgramData\anaconda3\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\PS User\AppData\Roaming\Python\Python311\site-packages\tkintermapview\map_widget.py", line 776, in mouse_move
    mouse_move_x = self.last_mouse_down_position[0] - event.x
                   ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^
TypeError: 'NoneType' object is not subscriptable
Exception in